In [2]:
import numpy as np
import pandas as pd
import json
import urllib
from datetime import datetime, timedelta, time
from sklearn import preprocessing

In [ ]:
# json 파일 읽어오기
start = datetime(2015,1,1) # 시작일
end = datetime(2021,1,1) # 종료일
my_dict = {'승차일자':list(),'시/도':list(), '시/군/구':list(), '동/읍/면':list(), '승차건수':list()}
while start != end:
    api = f'http://m.calltaxi.sisul.or.kr/api/open/newJSON0002.asp?key=e4bc2d5d397e041cc5f7a3fc3b41a150&sDate={start.strftime("%Y%m%d")}'
    page_obj = urllib.request.urlopen(api)
    json_page = json.loads(page_obj.read())
    for data in json_page['data']['list']:
        my_dict['승차일자'].append(data['useDate'])
        my_dict['시/도'].append(data['oc'])
        my_dict['시/군/구'].append(data['og'])
        my_dict['동/읍/면'].append(data['dong'])
        my_dict['승차건수'].append(data['cnt'])
    start = start + timedelta(days=1)
data = pd.DataFrame(my_dict)
display(data)

In [ ]:
destinationData = data.to_csv("./output/destinationDongData.csv", encoding = 'utf-8-sig')

In [110]:
destinationNum = pd.read_csv('destinationData.csv', encoding = 'utf-8-sig')
destinationNum['지역명'] = destinationNum['시/도'] + " " + destinationNum['시/군/구'] + " " + destinationNum['동']
destinationALL = destinationNum['승차건수'].groupby(destinationNum['동']).sum()

In [111]:
destinationdf = pd.DataFrame(destinationALL)
destinationALL = destinationdf.sort_values(by = '승차건수', ascending = False)
destinationALL.to_csv("destinationDongSort.csv", encoding = 'utf-8-sig')

In [5]:
dong = pd.read_csv('./dongData.csv') # 
dongSort = pd.read_csv('./dongSort.csv') # 승차건수 데이터

In [8]:
dongFinal1 = pd.merge(dong, dongSort, 'outer', on = '동')

,지역명,동,평균,승차건수
0,서울특별시 종로구 사직동,사직동,37.333333,8017.0
1,서울특별시 종로구 삼청동,삼청동,11.666667,63.0
2,서울특별시 종로구 부암동,부암동,42.166667,196.0
3,서울특별시 종로구 평창동,평창동,67.666667,1001.0
4,서울특별시 종로구 무악동,무악동,38.000000,58.0
...,...,...,...,...
559,NaN,광명2동,NaN,2.0
560,NaN,광명3동,NaN,2.0
561,NaN,호계3동,NaN,2.0
562,NaN,인창동,NaN,2.0


In [13]:
#결측치 삭제
dongFinal1 = dongFinal1.fillna(0); dongFinal1

,지역명,동,평균,승차건수
0,서울특별시 종로구 사직동,사직동,37.333333,8017.0
1,서울특별시 종로구 삼청동,삼청동,11.666667,63.0
2,서울특별시 종로구 부암동,부암동,42.166667,196.0
3,서울특별시 종로구 평창동,평창동,67.666667,1001.0
4,서울특별시 종로구 무악동,무악동,38.000000,58.0
...,...,...,...,...
559,0,광명2동,0.000000,2.0
560,0,광명3동,0.000000,2.0
561,0,호계3동,0.000000,2.0
562,0,인창동,0.000000,2.0


In [15]:
dongFinal1.to_csv("dongFinal1.csv", encoding = 'utf-8-sig') # 최종 여기에 항동만 더함

In [18]:
#완전 끝 = 데이터 정규화
final1 = pd.read_excel('./dongNor.xlsx')
x = final1.values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
final2 = pd.DataFrame(x_scaled, columns = final1.columns)

In [17]:
final2

,1,2
0,0.090229,0.058144
1,0.018233,0.000457
2,0.103787,0.001422
3,0.175316,0.007260
4,0.092099,0.000421
...,...,...
422,0.228612,0.327490
423,0.509584,0.038794
424,0.552127,0.152950
425,0.579243,0.237290


In [19]:
final2.to_csv('guNormalization.csv', encoding = 'utf-8-sig') # 최종 여기에 항동만 더함